In [1]:
import ads
import logging
import numpy as np
import os
import pandas as pd
import shutil
import tempfile
import warnings

from ads.dataset.dataset_browser import DatasetBrowser
from ads.dataset.factory import DatasetFactory
from ads.common.data import ADSData
from ads.common.model import ADSModel
from os import path
from sqlalchemy import create_engine
import seaborn as sns
from sklearn.base import TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import get_scorer
from collections import defaultdict
from ads.evaluations.evaluator import ADSEvaluator
#from ads.explanations.explainer import ADSExplainer

from ads.dataset.dataset_browser import DatasetBrowser
#from ads.feature_engineering import feature_type_manager, FeatureType
import matplotlib.pyplot as plt
from os import path

warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

In [2]:
print(ads.__version__)

2.8.8


In [3]:
diab = pd.read_csv('diabetes_prediction_dataset.csv')

In [4]:
diab_ds = DatasetFactory.from_dataframe(pd.DataFrame(diab), target = 'diabetes')

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
type(diab_ds)

ads.dataset.classification_dataset.BinaryClassificationDataset

In [7]:
diab_ds.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [11]:
diab_ds.shape

(100000, 9)

In [12]:
diab_ds.summary()

,Feature,Datatype
0,diabetes,categorical/int64
1,gender,categorical/object
2,age,continuous/float64
3,hypertension,categorical/int64
4,heart_disease,categorical/int64
5,smoking_history,categorical/object
6,bmi,continuous/float64
7,HbA1c_level,continuous/float64
8,blood_glucose_level,ordinal/int64


In [13]:
diab_ds.isna().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [14]:
diab_without_dup = diab_ds.drop_duplicates()
diab_without_dup.shape

(96146, 9)

In [15]:
diab_ds.show_in_notebook()

Accordion(children=(HTML(value='<h1>Name: User Provided DataFrame</h1><h3>Type: BinaryClassificationDataset</h…

In [18]:
diab_ds.get_recommendations()

Output()

In [21]:
diab_ds

diabetes,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,Female,13.000000,0,0,No Info,20.820000,5.800000,126
0,Female,3.000000,0,0,No Info,21.000000,5.000000,145
0,Male,63.000000,0,0,former,25.320000,3.500000,200
0,Female,2.000000,0,0,never,17.430000,6.100000,126
1,Female,33.000000,0,0,not current,40.080000,6.200000,200


BinaryClassificationDataset(target: diabetes) 100,000 rows, 9 columns

In [19]:
diab_ds.select_best_features()

diabetes,blood_glucose_level,HbA1c_level,age,bmi,hypertension,heart_disease,smoking_history,gender
0,126,5.800000,13.000000,20.820000,0,0,No Info,Female
0,145,5.000000,3.000000,21.000000,0,0,No Info,Female
0,200,3.500000,63.000000,25.320000,0,0,former,Male
0,126,6.100000,2.000000,17.430000,0,0,never,Female
1,200,6.200000,33.000000,40.080000,0,0,not current,Female


BinaryClassificationDataset(target: diabetes) 100,000 rows, 9 columns